# Cleaning Open311 requests from the city of Bonn

In [44]:
from three import Three
import pandas as pd

class BonnGeoReportServices(Three):
    def __init__(self):
        super(BonnGeoReportServices, self).__init__()
        self.endpoint = "http://anliegen.bonn.de/georeport/v2/"
        self.format = "json"
        self.jurisdiction = "anliegen.bonn.de"

geoReportServices = BonnGeoReportServices()
pageNumber = 0
userRequests = []
while True:
    results = geoReportServices.requests(page=pageNumber)
    pageNumber = pageNumber + 1
    requests = list(map(lambda request: { 'id':request.get('service_request_id'), 'code':request.get('service_code'), 'service':request.get('service_name'), 'lat':request.get('lat'), 'lon':request.get('long'), 'address':request.get('address'), 'date':request.get('requested_datetime'), 'text':request.get('description')}, results))
    if (requests[-1].get('id')):
        userRequests.extend(requests)
    else:
        requests = [request for request in requests if request.get('id')]
        userRequests.extend(requests)
        break

userRequestsDataFrame = pd.DataFrame.from_dict(userRequests)
#userRequestsDataFrame.to_csv('D:/OpenData.Bonn/open311-requests.tsv', sep='\t', index=False)

## Clean the user requests

In [42]:
from datacleaner import autoclean
import sys

try:
    cleanedUserRequestsDataFrame = autoclean(userRequestsDataFrame, copy=True)
except:
    print(sys.exc_info())
    userRequestsDataFrame.to_csv('D:/OpenData.Bonn/open311-invalid-autoclean.tsv', sep='\t', index=False)

(<class 'TypeError'>, TypeError("'<' not supported between instances of 'float' and 'str'",), <traceback object at 0x000002619E916D48>)


# Cleaning geonames

In [65]:
header = ['geonameid','name','asciiname','alternatenames','latitude','longitude','feature class','feature code','country code','cc2','admin1 code','admin2 code','admin3 code','admin4 code','population','elevation','dem','timezone','modification date']
geonames = pd.read_csv('D:/Geonames/cities1000_extract.txt', delimiter='\t', header=None, names=header, low_memory=False)
try:
    cleaned_geonames = autoclean(geonames, copy=True)
except:
    print(sys.exc_info())

In [77]:
combined_geonames = pd.concat([geonames, cleaned_geonames])
combined_geonames.reset_index(drop=True)
grouped_geonames = geonames.groupby(list(geonames.columns))
unique_index = [idx[0] for idx in grouped_geonames.groups.values() if len(idx) == 1]
differences_geonames = differences_geonames.reindex(unique_index)

print(geonames.loc[geonames['geonameid'] == 11592193])
print(cleaned_geonames.loc[cleaned_geonames['geonameid'] == 11592193])
#differences_geonames

      geonameid                 name            asciiname alternatenames  \
3774   11592193  Aschau im Zillertal  Aschau im Zillertal            NaN   

      latitude  longitude feature class feature code country code  cc2  \
3774  47.26613   11.89536             P        PPLA3           AT  NaN   

     admin1 code admin2 code admin3 code  admin4 code  population  elevation  \
3774          07         709       70902          NaN        1840        NaN   

      dem       timezone modification date  
3774  630  Europe/Vienna        2017-06-13  
      geonameid  name  asciiname  alternatenames  latitude  longitude  \
3774   11592193   427        447            6378  47.26613   11.89536   

      feature class  feature code  country code  cc2  admin1 code  \
3774              0             3            11    4           11   

      admin2 code  admin3 code  admin4 code  population  elevation  dem  \
3774         3067         1629      53053.0        1840      325.0  630   

      time